In [1]:
# Cell 5 (Fixed): Master Ingestion with Header Injection
import os
import glob
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:

# 1. Setup Embeddings
print("📥 Loading Hugging Face Embedding Model...")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. Define Splitter
headers_to_split_on = [
    ("#", "DocName"),
    ("##", "Section"),
    ("###", "SubSection"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

📥 Loading Hugging Face Embedding Model...


In [3]:
# 3. Process Files
all_splits = []
data_folder = "data"
md_files = glob.glob(os.path.join(data_folder, "*.md"))

print(f"📂 Found {len(md_files)} Markdown files.")

for file_path in md_files:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            file_content = f.read()
            
        splits = markdown_splitter.split_text(file_content)
        
        # --- THE FIX STARTS HERE ---
        for split in splits:
            # 1. Add Source Metadata
            split.metadata["source"] = file_path
            
            # 2. INJECT Metadata back into Content
            # We construct a "Context String" from the headers
            header_context = ""
            if "Section" in split.metadata:
                header_context += f"Section: {split.metadata['Section']}\n"
            if "SubSection" in split.metadata:
                header_context += f"Role/Topic: {split.metadata['SubSection']}\n"
            
            # Prepend it to the actual text
            split.page_content = header_context + split.page_content
        # --- THE FIX ENDS HERE ---
            
        all_splits.extend(splits)
        print(f"   ✅ Processed {os.path.basename(file_path)} -> {len(splits)} chunks.")
        
    except Exception as e:
        print(f"   ❌ Error reading {file_path}: {e}")

📂 Found 12 Markdown files.
   ✅ Processed about_college_accreditations.md -> 6 chunks.
   ✅ Processed academic_regulations.md -> 8 chunks.
   ✅ Processed admissions_process.md -> 12 chunks.
   ✅ Processed campus_facilities.md -> 7 chunks.
   ✅ Processed departments_overview.md -> 9 chunks.
   ✅ Processed eligibility_criteria.md -> 7 chunks.
   ✅ Processed fee_structure.md -> 8 chunks.
   ✅ Processed governance_and_contact.md -> 18 chunks.
   ✅ Processed hostel_transport.md -> 7 chunks.
   ✅ Processed placements_statistics.md -> 7 chunks.
   ✅ Processed reserach_innovation.md -> 4 chunks.
   ✅ Processed student_life.md -> 6 chunks.


In [4]:
# 4. Re-create Database (Clean Start)
import shutil
if os.path.exists("./chroma_db"):
    shutil.rmtree("./chroma_db")
    print("   🗑️  Cleared old database.")

print(f"⏳ Ingesting {len(all_splits)} chunks...")
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
    persist_directory="./chroma_db"
)
print("🎉 Database Updated! Headers are now visible to the AI.")

   🗑️  Cleared old database.
⏳ Ingesting 99 chunks...
🎉 Database Updated! Headers are now visible to the AI.


In [5]:
# Cell 11: The "Best of Both Worlds" - Hybrid Search
from langchain_classic.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

# 1. Setup Vector Retriever (The "Concept" Brain)
# k=3 finds general matches like "Dean of Research"
vector_retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# 2. Setup Keyword Retriever (The "Exact Match" Brain)
# We fetch all docs from the DB to build the keyword index
raw_docs = vectordb.similarity_search("dummy", k=100) 
bm25_retriever = BM25Retriever.from_documents(raw_docs)
bm25_retriever.k = 3

# 3. Create the Hybrid (Ensemble)
# weights=[0.5, 0.5] means we trust both brains equally
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.5, 0.5]
)

In [13]:

print("✅ Hybrid System Online: Vectors + Keywords")

# 4. FINAL TEST: "Tell me about Ranga"
query = "top companies for placments"
print(f"\n🔎 HYBRID SEARCH FOR: '{query}'")
print("="*60)

results = ensemble_retriever.invoke(query)

for i, doc in enumerate(results):
    print(f"🔹 Result #{i+1}")
    print("Source: ",doc.metadata['source'])
    # Check source to see which brain found it (BM25 usually finds names)
    print(f"   📄 Content: {doc.page_content}...")
    print("-" * 50)

✅ Hybrid System Online: Vectors + Keywords

🔎 HYBRID SEARCH FOR: 'top companies for placments'
🔹 Result #1
Source:  data\placements_statistics.md
   📄 Content: Section: 4. Top Recruiters (MNCs)
Major multi-national corporations and startups frequent the campus for recruitment drives.  
* **Tier 1 IT Services:** Infosys, TCS, Wipro, Cognizant (CTS), Tech Mahindra.
* **Major Recruiters:** Capgemini, Accenture, Hexaware, HCL.
* **Core & Startups:** Various core engineering firms and emerging startups also hire from relevant branches.
* **Search Tag:** **MNC, MNC's, retcutiers, companies, top companies, top MNC, top MNC's,jobs**  
---...
--------------------------------------------------
🔹 Result #2
Source:  data\placements_statistics.md
   📄 Content: Section: Important Disclaimer
Placement numbers often represent cumulative offers rather than unique students placed. Statistics are subject to market conditions and economic factors. For the absolute latest list of companies visiting this mo